# 0.8 Hierarchical Learning

## Boilerplate

The following subsections are largely boilerplate code, so skip around as needed.

### Jupyter Extensions

Load [watermark](https://github.com/rasbt/watermark) to see the state of the machine and environment that's running the notebook. To make sense of the options, take a look at the [usage](https://github.com/rasbt/watermark#usage) section of the readme.

In [1]:
# Load `watermark` extension
%load_ext watermark
# Display the status of the machine and packages. Add more as necessary.
%watermark -v -n -m -g -b -t -p numpy,matplotlib,seaborn,tensorflow

Mon Jun 03 2019 21:13:16 

CPython 3.6.8
IPython 7.3.0

numpy 1.16.2
matplotlib 3.0.2
seaborn 0.9.0
tensorflow 1.10.0

compiler   : GCC 7.3.0
system     : Linux
release    : 4.18.0-20-generic
machine    : x86_64
processor  : x86_64
CPU cores  : 8
interpreter: 64bit
Git hash   : d48242bb76989e2a67519e27e258f29143676920
Git branch : master


Load [autoreload](https://ipython.org/ipython-doc/3/config/extensions/autoreload.html) which will always reload modules marked with `%aimport`.

This behavior can be inverted by running `autoreload 2` which will set everything to be auto-reloaded *except* for modules marked with `%aimport`.

In [2]:
# Load `autoreload` extension
%load_ext autoreload
# Set autoreload behavior
%autoreload 1

Load `matplotlib` in one of the more `jupyter`-friendly [rich-output modes](https://ipython.readthedocs.io/en/stable/interactive/plotting.html). Some options (that may or may not have worked) are `inline`, `notebook`, and `gtk`.

In [3]:
# Set the matplotlib mode.
%matplotlib inline

### Imports

Static imports that shouldn't necessarily change throughout the notebook.

In [4]:
# Standard library imports
import logging
import os
from pathlib import Path
from copy import deepcopy
from pprint import pprint

# Third party
import IPython as ipy
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from pstar import pdict

Local imports that may or may not be autoreloaded. This section contains things that will likely have to be re-imported multiple times, and have additions or subtractions made throughout the project.

In [5]:
# Utility functions
%aimport leabratf.utils
from leabratf.utils import setup_logging
%aimport leabratf.constants
from leabratf.constants import DIR_DATA_PROC

### Initial Setup

Set [seaborn defaults](https://seaborn.pydata.org/generated/seaborn.set.html) for matplotlib.

In [6]:
sns.set()
sns.set_context("notebook")

Set up the logger configuration to something more useful than baseline. Creates log files for the different log levels in the `logs` directory.

See `logging.yml` for the exact logging configuration.

In [7]:
# Run base logger setup
setup_logging()
# Define a logger object
logger = logging.getLogger('leabratf')

## Task Definitions

### Constants

In [8]:
N_COLORS = 5
N_SHAPES = 4

### Phase Colors and Shapes

In [9]:
# All the colors and shapes
all_colors = [0, 1, 2, 3, 4]
all_shapes = [1 ,2, 3, 4]

# Phase A
phase_a_colors = [0, 1, 2]
phase_a_shapes = [1, 2]

# Phase B
phase_b_colors = [0, 1, 2]
phase_b_shapes = [3, 4]

# Phase C
phase_c_colors = [3, 4]
phase_b_shapes = [3, 4]

# Color lines correspond to a particular horizontal line
# # colors are not uniformly selected for
# Shapes corespond to a particular vertical line
# Color, Shape combinations correspond to a particular action 1-4

### Action Mapping


In [24]:
action_dictonary = {
    # (Color,Shape) : Action
    (0,1) : 1,
    (0,2) : 2,
    (0,3) : 1,
    (0,4) : 4,
    (1,1) : 1, 
    (1,2) : 2,
    (1,3) : 1,
    (1,4) : 4,
    (2,1) : 3,
    (2,2) : 4,
    (2,3) : 3,
    (2,4) : 2,
    (3,3) : 1,
    (3,4) : 3,
    (4,3) : 1,
    (4,4) : 2}

In [ ]:
def phase_a_labels(n_samples=100, 
                 colors=phase_a_colors,
                 shapes=phase_a_shapes,
                ):
    # N Color samples
    color_choices = np.eye(N_COLORS)[np.random.choice(
        phase_a_colors,
        size=n_samples,
        replace=True,
        p=[.25, .25, .5],
    )].reshape((n_samples, N_COLORS, 1))
    # N Shape samples
    shape_choices = np.eye(N_SHAPES)[np.random.choice(
        [s-1 for s in phase_a_shapes],
        size=n_samples,
        replace=True,
        p=[.5, .5],
    )].reshape((n_samples, N_SHAPES, 1))
    # Make the labels
    return color_choices, shape_choices

def integer_labels(labels):
    return [np.where(r==1)[0][0] for r in color_choices]
    
def input_array(color_choices, shape_choices):
    # Full Color array
    color_array = np.tile(color_choices, N_SHAPES)
    # Full Shape Array
    shape_array = np.transpose(
        np.tile(shape_choices, N_COLORS),
        [0, 2, 1])
        
    # Full data with both
    x_data = np.maximum(color_array, shape_array)
    return x_data

color_choices, shape_choices = phase_a_labels(10)
shape_choices    

In [26]:
def actions(color_choices, shape_choices):
    """Implements the mapping from shapes and colors to actions"""
    binarized = 
    label_tuples = [(color, shape) for color, shape in zip(
        color_choices, shape_choices)]
    return label_tuples
    
actions(color_choices, shape_choices)

[(array([[0.],
         [1.],
         [0.],
         [0.],
         [0.]]), array([[1.],
         [0.],
         [0.],
         [0.]])), (array([[0.],
         [0.],
         [1.],
         [0.],
         [0.]]), array([[0.],
         [1.],
         [0.],
         [0.]])), (array([[0.],
         [0.],
         [1.],
         [0.],
         [0.]]), array([[1.],
         [0.],
         [0.],
         [0.]])), (array([[0.],
         [1.],
         [0.],
         [0.],
         [0.]]), array([[0.],
         [1.],
         [0.],
         [0.]])), (array([[1.],
         [0.],
         [0.],
         [0.],
         [0.]]), array([[0.],
         [1.],
         [0.],
         [0.]])), (array([[0.],
         [1.],
         [0.],
         [0.],
         [0.]]), array([[1.],
         [0.],
         [0.],
         [0.]])), (array([[0.],
         [0.],
         [1.],
         [0.],
         [0.]]), array([[0.],
         [1.],
         [0.],
         [0.]])), (array([[0.],
         [1.],
         [0

In [21]:
[np.where(r==1)[0][0] for r in color_choices]

[1, 2, 2, 1, 0, 1, 2, 1, 2, 0]